In [6]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
conditions = {
    "alignment": ["aligned", "near-aligned", "far-aligned"],
    "anchor": ["anchor", "near-anchor", "far-anchor"],
    "scale": ["scale", "no-scale"],
    "round": ["round", "off-by-one", "off-by-two"]
}

# Create a list of all combinations of the conditions
from itertools import product
combinations = list(product(*conditions.values()))

# Remove non-round anchors
for combination in combinations:
    if combination[0] == "aligned" and combination[3] != "round":
        combinations.remove(combination)

combinations

[('aligned', 'anchor', 'scale', 'round'),
 ('aligned', 'anchor', 'scale', 'off-by-two'),
 ('aligned', 'anchor', 'no-scale', 'round'),
 ('aligned', 'anchor', 'no-scale', 'off-by-two'),
 ('aligned', 'near-anchor', 'scale', 'round'),
 ('aligned', 'near-anchor', 'scale', 'off-by-two'),
 ('aligned', 'near-anchor', 'no-scale', 'round'),
 ('aligned', 'near-anchor', 'no-scale', 'off-by-two'),
 ('aligned', 'far-anchor', 'scale', 'round'),
 ('aligned', 'far-anchor', 'scale', 'off-by-two'),
 ('aligned', 'far-anchor', 'no-scale', 'round'),
 ('aligned', 'far-anchor', 'no-scale', 'off-by-two'),
 ('near-aligned', 'anchor', 'scale', 'round'),
 ('near-aligned', 'anchor', 'scale', 'off-by-one'),
 ('near-aligned', 'anchor', 'scale', 'off-by-two'),
 ('near-aligned', 'anchor', 'no-scale', 'round'),
 ('near-aligned', 'anchor', 'no-scale', 'off-by-one'),
 ('near-aligned', 'anchor', 'no-scale', 'off-by-two'),
 ('near-aligned', 'near-anchor', 'scale', 'round'),
 ('near-aligned', 'near-anchor', 'scale', 'off-by

In [8]:
anchors = [25, 50]
nearAnchors = [21, 22, 23, 24, 26, 27, 28, 29, 46, 47, 48, 49]
startPositions = [0, 25, 50, 75]
endPositions = [25, 50, 75, 100]

values = []
alignmentCategories = []
alignmentDistances = []
alignmentTypes = []
valueAlignments = []
anchorCategories = []
anchorDistances = []
round5Distances = []
positions = []

for selectedPart in range(15, 56):
    for startPosition in range(0, 100):
        endPosition = startPosition + selectedPart # calculate end position

        # remove impossible positions
        if endPosition > 100:
            continue

        # calculate minimum distance from alignment for both ends
        startDists = [startPosition, abs(startPosition - 25), abs(startPosition - 50), abs(startPosition - 75)]
        endDists = [abs(endPosition - 25), abs(endPosition - 50), abs(endPosition - 75), abs(endPosition - 100)]
        
        startAlignment = np.min(startDists)
        endAlignment = np.min(endDists)

        # determine which end is closer to alignment
        if startAlignment < endAlignment:
            alignmentSide = "start"
            alignmentPosition = startPositions[np.argmin(startDists)]
        elif startAlignment > endAlignment:
            alignmentSide = "end"
            alignmentPosition = endPositions[np.argmin(endDists)]
        else:
            alignmentSide = "both ends"
            startAligned = np.argmin(startDists)
            endAligned = np.argmin(endDists)
            if startAligned < endAligned:
                alignmentPosition = f"{startPositions[startAligned]} and {endPositions[endAligned]}"
            else:
                alignmentPosition = f"{startPositions[startAligned]} and {endPositions[endAligned]}"

        # determine the alignment category
        alignmentDistance = min(startAlignment, endAlignment)
        if alignmentDistance == 0:
            alignmentCategory = "aligned"
        elif alignmentDistance < 5:
            alignmentCategory = "near-aligned"
        else: 
            alignmentCategory = "far-from-aligned"

        # determine anchor category
        anchorDistance = min(abs(selectedPart - 25), abs(selectedPart - 50))
        if selectedPart in anchors:
            anchorCategory = "anchor"
        elif selectedPart in nearAnchors:
            anchorCategory = "near-anchor"
        else:
            anchorCategory = "far-anchor"

        # determine distance to the nearest 5
        round5Distance = abs(selectedPart - round(selectedPart / 5) * 5)

        # describe every possible selectedPart/alignment condition in the range
        valueAlignment = f"Value {selectedPart} with {alignmentSide} {alignmentDistance} off alignment with {alignmentPosition}."
        if valueAlignment not in valueAlignments:
            valueAlignments.append(valueAlignment)
            values.append(selectedPart)
            alignmentCategories.append(alignmentCategory)
            anchorCategories.append(anchorCategory)
            alignmentDistances.append(alignmentDistance)
            anchorDistances.append(anchorDistance)
            round5Distances.append(round5Distance)
            alignmentTypes.append(alignmentPosition)
            positions.append(startPosition)

df = pd.DataFrame({
    # "Description": valueAlignments,
    "selectedPart": values,
    "alignmentCategory": alignmentCategories,
    "anchorCategory": anchorCategories,
    "anchorDistance": anchorDistances,
    "alignmentDistance": alignmentDistances,
    "round5Distance": round5Distances,
    "alignmentType": alignmentTypes,
    "alignmentPosition": positions
})
df.to_csv("data/allConditions.csv", index=False)

In [9]:
df

,selectedPart,alignmentCategory,anchorCategory,anchorDistance,alignmentDistance,round5Distance,alignmentType,alignmentPosition
0,15,aligned,far-anchor,10,0,0,0,0
1,15,near-aligned,far-anchor,10,1,0,0,1
2,15,near-aligned,far-anchor,10,2,0,0,2
3,15,near-aligned,far-anchor,10,3,0,0,3
4,15,near-aligned,far-anchor,10,4,0,0,4
...,...,...,...,...,...,...,...,...
2297,55,near-aligned,far-anchor,5,4,0,100,41
2298,55,near-aligned,far-anchor,5,3,0,100,42
2299,55,near-aligned,far-anchor,5,2,0,100,43
2300,55,near-aligned,far-anchor,5,1,0,100,44


In [10]:
df = pd.read_csv("data/pilotStimuli.csv")
df

,selectedPart,alignmentCategory,anchorCategory,anchorDistance,alignmentDistance,round5Distance,alignmentType,alignmentPosition,chartType,UID,selectedLabel
0,50,aligned,anchor,0,0,0,50 and 100,50,line,1,F
1,50,aligned,anchor,0,0,0,50 and 100,50,line,1,D
2,50,near-aligned,anchor,0,3,0,0 and 50,3,line,1,A
3,25,near-aligned,anchor,0,3,0,75 and 100,72,line,1,A
4,25,near-aligned,anchor,0,3,0,0 and 25,3,line,1,E
...,...,...,...,...,...,...,...,...,...,...,...
1915,51,far-from-aligned,far-anchor,1,6,1,0,6,pie,20,A
1916,40,far-from-aligned,far-anchor,10,5,0,0 and 50,5,pie,20,C
1917,30,far-from-aligned,far-anchor,5,9,0,50,59,pie,20,B
1918,45,far-from-aligned,far-anchor,5,6,0,50,44,pie,20,G
